# Key Value Store

Values are commonly stored with a key that is used to access the value from the store. This package defines a class that makes it simple to expose keyed-values as properties.

## Example

Define an implementation of the KeyValueStore. This one merely wraps the standard `dict`.

In [1]:
from keyvaluestore import KeyValueStore

class Store(KeyValueStore):
    values: dict

    def __init__(self, key_values: dict = None):
        self.values = {}
        if key_values is not None:
            self.values.update(key_values)

    def get(self, keys: list or str = None, fallback = None):
        """
        This getter enables multiple keys to be retrieved at a time.
        """
        if keys is None:
            keys = list(self.values.keys())

        if isinstance(keys, list):
            return {
                key: self.values.get(key, fallback)
                for key in keys
            }
        else:
            return self.values.get(keys, fallback)

    def set(self, keys: str or list, values):
        """
        This getter enables multiple keys to be retrieved at a time.
        """
        if isinstance(keys, list):
            for index, key in enumerate(keys):
                self.values[key] = values[index]
        else:
            self.values[keys] = values

Add properties to the implemented class `Store` with various behaviours.

In [2]:
from keyvaluestore import KeyValueProperty

KeyValueStore.add_properties(
    Store,
    [
        KeyValueProperty(
            name="string",
            key="string",
            # invoke the default getter/setter by not specifying them
            documentation="This value is keyed by 'string' and has the default getter/setter of the class."
        ),
        KeyValueProperty(
            name="string_length",
            key=None, # No key provided as the value is computed, not stored
            getter=lambda self: len(self.get("string", "")), # self is an instance of the class
            setter=None, # indicate that the property cannot be set
            documentation="This value is keyed by 'string_length' and is a read-only, computed value."
        ),
        KeyValueProperty(
            name="asterisks",
            key="asterisks", # No key provided as the value is computed, not stored
            # invoke the default getter by not specifying it
            setter=lambda self, count: self.set("asterisks", "*"*int(count)), # self is an instance of the class
            documentation="This value is keyed by 'asterisks' and has a bespoke ."
        ),
    ]
)

Now excercise an instance of the `Store`.

In [3]:
store = Store()
# Get `asterisks` property before setting
store.asterisks == None

True

In [4]:
store.asterisks = 4
store.asterisks

'****'

In [5]:
# Set the `string` property
store.string = "Mic-check: 1, 2, 3."
store.string

'Mic-check: 1, 2, 3.'

In [6]:
# Get the associated `string_length` property
store.string_length

19

In [7]:
try:
    store.string_length = 4
    raise Exception("Unexpected success!")
except AttributeError as err:
    print(f"As expected: {err}")

As expected: can't set attribute


In [8]:
store.set(["string", "asterisks"], ["**Applause**", 8])
store.get(["string", "asterisks"])

{'string': '**Applause**', 'asterisks': 8}

In [9]:
store.get()

{'asterisks': 8, 'string': '**Applause**'}

In [10]:
print(store)

<class '__main__.Store'>: {'asterisks': 8, 'string': '**Applause**', 'string_length': 12}
